In [280]:
import pandas as pd
import numpy as np
import itertools

input_file = 'input_example.txt'
# input_file = 'input_full.txt'

with open(input_file) as file:
    lines = file.readlines()
    lines = [line.strip() for line in lines]

# Identify all starts & ends of rocks
rocks = []
for line in lines:
    rocks += [[int(coord) for coord in coord.split(',')] for coord in line.replace(' -> ', ';').split(';')]
# Identify the rectangle where all rocks are contained
max_x, min_x = np.array(rocks)[:,0].max(), np.array(rocks)[:,0].min()
max_y = np.array(rocks)[:,1].max()
# Extend this space so that it can contain a pyramid from (500, 0)
min_x, max_x = min(min_x, 500-max_y-1), max(max_x, 500+max_y+1)

# Create a dataframe representing the cave map: 0 means air, 1 means rock and 2 means sand
# Initialize columns
df = pd.DataFrame(columns=[x for x in range(min_x-1, max_x+2)])
# Create the rows
for y in range(max_y+1):
    df.loc[len(df)] = '.'
# For Part 1, add an additional empty row at the bottom (representing the infinite space)
df.loc[len(df)] = '.'
# For Part 2, add an additional rock row at the bottom
df.loc[len(df)] = '#'
# Insert rocks into the map
for line in lines:
    rocks = [[int(coord) for coord in coord.split(',')] for coord in line.replace(' -> ', ';').split(';')]
    
    for i in range(1, len(rocks)):
        rock_from, rock_to = rocks[i-1], rocks[i]
        rock_min_x, rock_max_x = min(rock_from[0], rock_to[0]), max(rock_from[0], rock_to[0])
        rock_min_y, rock_max_y = min(rock_from[1], rock_to[1]), max(rock_from[1], rock_to[1])
        for (x, y) in itertools.product(range(rock_min_x, rock_max_x+1), range(rock_min_y, rock_max_y+1)):
            df.at[y, x] = '#'
pd.set_option('display.max_columns', 25)
df

,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
2,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
3,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
4,.,.,.,.,.,.,.,.,.,#,.,.,.,#,#,.,.,.,.,.,.,.,.
5,.,.,.,.,.,.,.,.,.,#,.,.,.,#,.,.,.,.,.,.,.,.,.
6,.,.,.,.,.,.,.,#,#,#,.,.,.,#,.,.,.,.,.,.,.,.,.
7,.,.,.,.,.,.,.,.,.,.,.,.,.,#,.,.,.,.,.,.,.,.,.
8,.,.,.,.,.,.,.,.,.,.,.,.,.,#,.,.,.,.,.,.,.,.,.
9,.,.,.,.,.,#,#,#,#,#,#,#,#,#,.,.,.,.,.,.,.,.,.


In [281]:
[sand_initial_x, sand_initial_y] = [500, 0]

# Function to calculate the next move of a sand
def move(sandx, sandy, part=1):
    global df_map
    global max_y
    if (part == 1) and (sandy > max_y):
        return None
    elif df_map.at[sandy+1, sandx] == '.':
        return move(sandx, sandy+1, part)
    elif df_map.at[sandy+1, sandx-1] == '.':
        return move(sandx-1, sandy+1, part)
    elif df_map.at[sandy+1, sandx+1] == '.':
        return move(sandx+1, sandy+1, part)
    else:
        return [sandx, sandy]

In [282]:
# Part 1
df_map = df.copy()

# Simulate each sand (there cannot be more sand than the shape of the map)
for i in range(1, df_map.shape[0]*df_map.shape[1]):
    sand_final_coord = move(sand_initial_x, sand_initial_y, part=1)
    if sand_final_coord is not None:
        sand_final_x, sand_final_y = sand_final_coord
        df_map.at[sand_final_y, sand_final_x] = 'o'
    else:
        print(f'Part 1: Sand N.{i} is falling for ever, but sand N.{i-1} landed at {(sand_final_x, sand_final_y)}')
        break
df_map

Part 1: Sand N.25 is falling for ever, but sand N.24 landed at (495, 8)


,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
2,.,.,.,.,.,.,.,.,.,.,.,o,.,.,.,.,.,.,.,.,.,.,.
3,.,.,.,.,.,.,.,.,.,.,o,o,o,.,.,.,.,.,.,.,.,.,.
4,.,.,.,.,.,.,.,.,.,#,o,o,o,#,#,.,.,.,.,.,.,.,.
5,.,.,.,.,.,.,.,.,o,#,o,o,o,#,.,.,.,.,.,.,.,.,.
6,.,.,.,.,.,.,.,#,#,#,o,o,o,#,.,.,.,.,.,.,.,.,.
7,.,.,.,.,.,.,.,.,.,o,o,o,o,#,.,.,.,.,.,.,.,.,.
8,.,.,.,.,.,.,o,.,o,o,o,o,o,#,.,.,.,.,.,.,.,.,.
9,.,.,.,.,.,#,#,#,#,#,#,#,#,#,.,.,.,.,.,.,.,.,.


In [284]:
# Part 2
df_map = df.copy()

for i in range(1, df_map.shape[0]*df_map.shape[1]):
    [sand_final_x, sand_final_y] = move(sand_initial_x, sand_initial_y, part=2)
    df_map.at[sand_final_y, sand_final_x] = 'o'
    if [sand_final_x, sand_final_y] == [sand_initial_x, sand_initial_y]:
        print(f'Part 2: Sand N.{i} cannot move down')
        break
df_map

Part 2: Sand N.93 cannot move down


,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,.,.,.,.,.,.,.,.,.,.,.,o,.,.,.,.,.,.,.,.,.,.,.
1,.,.,.,.,.,.,.,.,.,.,o,o,o,.,.,.,.,.,.,.,.,.,.
2,.,.,.,.,.,.,.,.,.,o,o,o,o,o,.,.,.,.,.,.,.,.,.
3,.,.,.,.,.,.,.,.,o,o,o,o,o,o,o,.,.,.,.,.,.,.,.
4,.,.,.,.,.,.,.,o,o,#,o,o,o,#,#,o,.,.,.,.,.,.,.
5,.,.,.,.,.,.,o,o,o,#,o,o,o,#,o,o,o,.,.,.,.,.,.
6,.,.,.,.,.,o,o,#,#,#,o,o,o,#,o,o,o,o,.,.,.,.,.
7,.,.,.,.,o,o,o,o,.,o,o,o,o,#,o,o,o,o,o,.,.,.,.
8,.,.,.,o,o,o,o,o,o,o,o,o,o,#,o,o,o,o,o,o,.,.,.
9,.,.,o,o,o,#,#,#,#,#,#,#,#,#,o,o,o,o,o,o,o,.,.
